# M1: Aspiradora
## Nombre: Pablo Emilio García Zertuche
## Matricula: A00831264

### 1er paso, hacer los imports.
- Agent y Model nos ayudan a que funciona el piso (ya sea sucia o limpia) y las aspiradoras
- MultiGrid nos ayuda a definir que se pueden tener varios agentes en la misma ubicacion, ya sea agente piso o aspiradora/s

In [2]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

### 2ndo paso, hacer los agentes y el model

In [3]:
def obtener_habitacion(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        
        for contenido in contenido_celda:
            if isinstance(contenido, Aspiradora):
                habitacion[x][y] = 2
            else:
                habitacion[x][y] = contenido.estado
                
    return habitacion

class Aspiradora(Agent):
    
    """
    Variables que se necesitan:
        -cantidad de movimientos
        -posicion de la aspiradora
    """
    
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.nueva_posicion = None
        self.movimientos = 0
    
    def step(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore = True,
            include_center = True)
        
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos:
                vecino.siguiente_estado = vecino.estado
                if(vecino.siguiente_estado == vecino.SUCIO):
                    vecino.siguiente_estado = vecino.LIMPIO
                    self.nueva_posicion = self.pos
                else:
                    vecindario = self.model.grid.get_neighborhood(
                        self.pos,
                        moore = True,
                        include_center = False)
                    nueva_posicion = self.random.choice(vecindario)
                    self.nueva_posicion = nueva_posicion
                break
    
    def advance(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore = False,
            include_center = True)
        
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos:
                vecino.estado = vecino.siguiente_estado
                break
        
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1
        
        self.model.grid.move_agent(self, self.nueva_posicion)

class Piso(Agent):
    SUCIO = 1
    LIMPIO = 0
    
    def __init__(self, pos, modelo, estado=LIMPIO):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.estado = estado
        self.siguiente_estado = None

class Habitacion(Model):
    def __init__(self, m, n, num_agentes, por_celdas_sucias):
        self.num_agentes = num_agentes
        self.por_celdas_sucias = por_celdas_sucias
        self.por_celdas_limpias = 1 - por_celdas_sucias
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
        
        celdas_sucias = int((m * n) * por_celdas_sucias)
        lista_celdas_vacias = list(self.grid.empties)
        
        for celdas in range(celdas_sucias):
            celda_vacia = random.choice(lista_celdas_vacias)
            piso = Piso(celda_vacia, self)
            piso.estado = piso.SUCIO
            self.grid.place_agent(piso, celda_vacia)
            self.schedule.add(piso)
            lista_celdas_vacias.remove(celda_vacia)
        
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in lista_celdas_vacias:
            piso = Piso(celdas, self)
            self.grid.place_agent(piso, celdas)
            self.schedule.add(piso)
            
        
        for i in range(num_agentes):
            aspiradora = Aspiradora(i, self)
            self.grid.place_agent(aspiradora, (0,0))
            self.schedule.add(aspiradora)
        
        self.colectordatos = DataCollector(
            model_reporters = {'Habitacion': obtener_habitacion},
            agent_reporters = {'Movimientos': lambda a: getattr(a,'movimientos', None)}
        )
        
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()
        
    def todasceldaslimpias(self):
        celdas_limpias = 0
        for celda in self.grid.coord_iter():
            contenido_celda, x, y = celda
            for contenido in contenido_celda:
                if isinstance(contenido, Piso) and contenido.estado == contenido.LIMPIO:
                    celdas_limpias = celdas_limpias + 1
        
        self.por_celdas_limpias = celdas_limpias / (self.grid.width * self.grid.height)
        
        if self.por_celdas_limpias == 1:
            return True
        else:
            return False
        
            

### 3er paso, correr el modelo

In [4]:
#Datos de la habitacion
M = 15
N = 15

#Numero de agentes
NUM_AGENTES = 10

# Porcentaje de celdas sucias
PORCENTAJE_CELDAS_SUCIAS = 0.5

# Tiempo máximo de ejecución
TIEMPO_MAXIMO_EJECUCION = 0.06

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(M, N, NUM_AGENTES, PORCENTAJE_CELDAS_SUCIAS)
while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasceldaslimpias()):
    modelo.step()

tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

## Visualización 

In [5]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()

In [6]:
%%capture

fig, axs = plt.subplots(figsize = (7,7))
axs.set_xticks([])
axs.set_yticks([])

patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap = 'Greys')

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

In [7]:
anim

## Informe

In [8]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que todas las celdas estén limpias: ', tiempo_ejecucion, '/', tiempo_inicio)
print('Porcentaje de celdas limpias después del termino de la solución: ', round(modelo.por_celdas_limpias * 100, 2), '%')
print('Número de movimientos realizados por todos los agentes: ', int(movimientos.tail()['Movimientos'].sum()))

Tiempo necesario hasta que todas las celdas estén limpias:  0:00:00.060108 / 0:00:00.060000
Porcentaje de celdas limpias después del termino de la solución:  72.0 %
Número de movimientos realizados por todos los agentes:  111


Es importante notar que en este codigo los robots que aspiran se transportan aleatoriamente, por lo que significa que no hay una estrategia o comunicación entre los robotos. Una manera simple de solucionar este problema es la parte de colisiones entre los robots, ya que de este modo no habria manera de 2 o más robots existan en una celda al mismo tiempo. Asimismo, al momento de realizar un movimeinto, verificar si hay basura dentro del perimetro del robot para en lugar de moverse a una ubicación aleatoria este robot limpia. No obstante, en solo 111 movimientos estos alcanzaron el 72.0% de limpieza, algo que sigue siendo efectivo, sin embargo, se ocuparía el 100%. Por esto mismo, se aumento la variable de la cantidad de agentes para visualizar la eficiencia de los robots:

In [25]:
#Datos de la habitacion
M = 15
N = 15

#Numero de agentes
NUM_AGENTES = 20

# Porcentaje de celdas sucias
PORCENTAJE_CELDAS_SUCIAS = 0.5

# Tiempo máximo de ejecución
TIEMPO_MAXIMO_EJECUCION = 0.06

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(M, N, NUM_AGENTES, PORCENTAJE_CELDAS_SUCIAS)
while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasceldaslimpias()):
    modelo.step()

tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

In [26]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()

In [27]:
%%capture

fig, axs = plt.subplots(figsize = (7,7))
axs.set_xticks([])
axs.set_yticks([])

patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap = 'Greys')

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

In [28]:
anim

In [29]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que todas las celdas estén limpias: ', tiempo_ejecucion, '/', tiempo_inicio)
print('Porcentaje de celdas limpias después del termino de la solución: ', round(modelo.por_celdas_limpias * 100, 2), '%')
print('Número de movimientos realizados por todos los agentes: ', int(movimientos.tail()['Movimientos'].sum()))

Tiempo necesario hasta que todas las celdas estén limpias:  0:00:00.061099 / 0:00:00.060000
Porcentaje de celdas limpias después del termino de la solución:  98.22 %
Número de movimientos realizados por todos los agentes:  295


En esta ocasion se aumentó la cantidad de robots a 20 y se obtuvo un resultado de 98.22%, un brinco drástico al resultado anterior con solo 10 robots de 72%. Cabe mencionar que se aumentaron un 200% la cantidad de robots y como quiera no se obtuvo el resultado meta de 100%. Esto afecta en gran medida al proyecto, debido a que si se vendería, no se podría garantizar con certeza que su habitación quede limpia al 100%. Adicionalmente, se le tuviera que advertir al cliente de comprar más productos para poder limpiar su cuarto, lo cual no es lo optimo. Por otro lado, se vio un brinco significativo en los movimientos, debido a que en teoría si se realizaron 111 movimientos con 10 robots, este debería de tener 222, no obstante, se movieron 295. Lo importante a notar es como la cantidad de robots es sumamente importante para la efectividad en dicho tiempo, y fluctuar este parametro impacta la efectividad del resultado.

Por otro lado, otra manera de solucionar este problema sin tener que aumentar los robots, es tener un claro mapeo de la congestion de basura dentro de la habitación, con el fin de dirigir una mayor cantidad de robots en esa dirección para rápidamente limpiar la habitación. Para esto se debería de visualizar si hay mas de una casilla con basura y preguntar si hay mas robots cerca del perimetro, este con el fin de limpiar dos o más casillas en un movimiento. 

En conclusión, este modelo por si solo ocupa una mejora para poder realizar sus tareas más eficientes, por ejemplo, la comunicación entre los robots para evitar movimientos redundantes y para realizar un mapeo de conjunto de basura. Por otro lado, es importante mencionar que hay muchos factores que afectan la efectividad del modelo, por lo que ahorita no se tiene con certeza cuál es el mejor metodo y el control de las variables. Es por esto que es importante probar el modelo para buscar mejoras para poder probeer a cualquiera el mejor resultado.